In [1]:
from scrapping.images import get_images
from scrapping.box import get_box_labels

get_images()

In [2]:
get_box_labels()

2024-02-12 00:20:15.204 | INFO     | scrapping.box:get_box_labels:35 - Created OCR
2024-02-12 00:20:15.207 | INFO     | scrapping.box:get_box_labels:37 - Streams: ['distortion2', 'summit1g', 'zackrawrr']
2024-02-12 00:20:15.207 | INFO     | scrapping.box:get_box_labels:39 - Getting labels of distortion2
2024-02-12 00:20:15.208 | INFO     | scrapping.box:get_streamer_labels:13 - Images of streamer distortion2: ['cropped_image_0.png', 'cropped_image_1.png', 'cropped_image_10.png', 'cropped_image_11.png', 'cropped_image_2.png', 'cropped_image_3.png', 'cropped_image_4.png', 'cropped_image_5.png', 'cropped_image_6.png', 'cropped_image_7.png', 'cropped_image_8.png', 'cropped_image_9.png']
2024-02-12 00:20:15.210 | INFO     | scrapping.box:get_streamer_labels:18 - Using batch size of 10
2024-02-12 00:20:15.211 | INFO     | scrapping.box:get_streamer_labels:20 - Amount of batches: 2
2024-02-12 00:20:15.211 | INFO     | scrapping.box:get_streamer_labels:24 - Streamer: distortion2, Batch: ['data

In [ ]:
from os import listdir
import pickle
import easyocr

from conf.settings import Settings

In [ ]:
settings = Settings()

In [ ]:
reader = easyocr.Reader(['en'],cudnn_benchmark=True)

In [ ]:
streams = listdir(settings.cropped_folder)

In [ ]:
def get_streamer_labels(streamer,batch_size):
    images = listdir(f"{settings.cropped_folder}/{streamer}")    
    if '.ipynb_checkpoints' in images:
        images.remove('.ipynb_checkpoints')    
    images_folder = [f"{settings.cropped_folder}/{streamer}/{image}" for image in images]

    batches = [images_folder[i:i+batch_size] for i in range(0, len(images_folder), batch_size)]
    streamer_result = {}
    for batch in batches:  
        batch_result = reader.readtext_batched(batch)    
        result_dict = {key: value for key, value in zip(batch, batch_result)}
        streamer_result.update(result_dict)
    return streamer_result

In [ ]:
ok = get_streamer_labels(streams[0],5)

In [ ]:
with open("prueba", "wb") as fp:   
    pickle.dump(result, fp)